In [1]:
"""
Things to add: 
1) Make sure neighbors have a shared edge and not just shared vertices (think we are good tho) because worried about being manifold later on
- Because no instance where 2 faces share vertices and pretty close in normal to where the neighbor of both of them (that is between) wouldn't be included in the facet
 --->  but could do these just to be safe
 
Solution to manifold:
1) if we find a pinch point --> will lead to non-manifold connection structure possible
2) If using a pinch point face:
a. Add all of vertex neighbors to the facet group --> recompute the boundary and again make sure no pitch points
 --> keep doing this until no pinch points
 
How to tell if there is:
Wi
 
"""


"\nThings to add: \n1) Make sure neighbors have a shared edge and not just shared vertices (think we are good tho) because worried about being manifold later on\n- Because no instance where 2 faces share vertices and pretty close in normal to where the neighbor of both of them (that is between) wouldn't be included in the facet\n --->  but could do these just to be safe\n \nSolution to manifold:\n1) if we find a pinch point --> will lead to non-manifold connection structure possible\n2) If using a pinch point face:\na. Add all of vertex neighbors to the facet group --> recompute the boundary and again make sure no pitch points\n --> keep doing this until no pinch points\n \nHow to tell if there is:\nWi\n \n"

In [2]:
import trimesh
import numpy as np
from collections import Counter
import time
import math

In [3]:
#Example mesh to practice stitching on 

vertices = np.array([
    [0,0,7], #0
    [10,0,0], #1
    [0,10,0], #2
    [-10,0,0], #3
    [0,-10,0], #4
    
    [5,0,7], #5
    [5,5,7], #6
    [0,5,7], #7
    [-5,5,7], #8
    [-5,0,7], #9
    [-5,-5,7], #10
    [0,-5,7], #11
    [5,-5,7] ##12
    
])

faces = np.array([
    [1,2,3],
    [1,3,4],
    
    [0,5,6],
    [0,6,7],
    [0,7,8],
    [0,8,9],
    [0,9,10],
    [0,10,11],
    [0,11,12],
    [0,12,5]

    
])



In [4]:
new_mesh = trimesh.Trimesh()
new_mesh.faces = faces
new_mesh.vertices = vertices
new_mesh.show()

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [5]:
#retrieving correctly oriented versions of the normals

#how to find the normals of facet groups:
facet_group_1_normal = new_mesh.face_normals[0]
facet_group_2_normal = new_mesh.face_normals[2]*-1


#get the correct version of the normals:
if np.dot(facet_group_1_normal,facet_group_2_normal) > 0.8:
    pass
elif np.dot(facet_group_1_normal,facet_group_2_normal) < -0.8:
    #flip the first normal
    facet_group_1_normal = facet_group_1_normal*-1
else:
    print("Not correct normals")

print(facet_group_1_normal,facet_group_2_normal)

[-0. -0. -1.] [-0. -0. -1.]


In [6]:
"""
Gets unordered boundary edges of each of the facets
"""

# make each row correspond to a single face
edges = new_mesh.edges_sorted.reshape((-1, 6))
# get the edges for each facet
edges_facet = [edges[i].reshape((-1, 2)) for i in new_mesh.facets]
edges_boundary = np.array([i[trimesh.grouping.group_rows(i, require_count=1)]
                           for i in edges_facet])

edges_boundary #has both of the groups of edges

array([array([[1, 2],
       [2, 3],
       [1, 4],
       [3, 4]]),
       array([[ 5,  6],
       [ 6,  7],
       [ 7,  8],
       [ 8,  9],
       [ 9, 10],
       [10, 11],
       [ 5, 12],
       [11, 12]])], dtype=object)

In [92]:
edges_boundary.shape

(2,)

In [7]:
"""
How to detect and fix pinch points

1) compute the boundary
2) Look for any points with more than 2 edges (in the boundary)
3) How do you know where the gaps are from the edges
4) Get all the vertices neighbors of the pinch points:
a. Could need a 

"""

'\nHow to detect and fix pinch points\n\n1) compute the boundary\n2) Look for any points with more than 2 edges (in the boundary)\n3) How do you know where the gaps are from the edges\n4) Get all the vertices neighbors of the pinch points:\na. Could need a \n\n'

In [8]:
#gets the projection of point p onto line a
def ClosestPointOnLine(a, b, p):
    ap = p-a
    ab = b-a
    #base_vector = ab
    result = np.dot(ap,ab)/np.dot(ab,ab) # * ab
    return result

In [9]:
"""
gets the corresponding starting indices that meshing process will start 
"""
#def get_staring_index(edges_boundary):
#get all of the unique points for each edge

edges_boundary
edge_0 = edges_boundary[0]
edge_1 = edges_boundary[1]

#gets the unique number of points
edge_0_points = np.unique(np.hstack(edge_0))
edge_1_points = np.unique(np.hstack(edge_1))

"""
get the dot product of all the points
"""

#get any 2 points on the triangle and make that the reference traingle
edge_0_anchor_points = new_mesh.vertices[[edge_0_points[0],edge_0_points[1]]]

#gets the starting index for the 1st facet
max_index = edge_0_points[0]
max_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[max_index])

for i in range(1,len(edge_0_points)):
    current_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[edge_0_points[i]])
    
    if current_magnitude > max_magnitude:
        max_index = i
        max_magnitude = current_magnitude

edge_0_starting_point = edge_0_points[max_index]

#gets the starting index for the 2nd facet
max_index = edge_1_points[0]
max_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[max_index])

for i in range(1,len(edge_1_points)):
    current_magnitude = ClosestPointOnLine(edge_0_anchor_points[0],edge_0_anchor_points[1],new_mesh.vertices[edge_1_points[i]])
    if current_magnitude > max_magnitude:
        max_index = i
        max_magnitude = current_magnitude

edge_1_starting_point = edge_1_points[max_index]

print(f"starting edge 1st facet = {edge_0_starting_point}, starting edge 2nd facet= {edge_1_starting_point}, ")
print(new_mesh.vertices[edge_0_starting_point],new_mesh.vertices[edge_1_starting_point])
        

starting edge 1st facet = 2, starting edge 2nd facet= 8, 
[ 0. 10.  0.] [-5.  5.  7.]


In [10]:
"""
Need to order the points for restitching

Pseudocode: 
1) Get starting piont
2) Find the two edges corresponding to that point
3) Need to decide which direction to start....
- go in direction that make the cross of the (1st and last) point in the same direction of the 
normal of the first facet
4) loop through and record the orders of the vertices as you traverse along the edges 
until you arrive back at the start
5) Error if:
    a. You arrive back at the start and haven't processed all the edges
    b. Processsed all the edges but haven't arrived back at start
    
6) Repeat steps 1 through 5 for 2nd facet group
"""
start_point_list = [edge_0_starting_point,edge_1_starting_point]
edge_list = [edge_0,edge_1]
edge_order_list = []


for i,start_point in enumerate(start_point_list):
    print(f"start_point = {start_point}")
    edge_order = [start_point]
    processed_edges = []

    #find the matching edges
    starting_edges_indices = np.where(np.logical_or(edge_list[i][:,0] == start_point,edge_list[i][:,1] == start_point) == True)[0]
    #print(starting_edges_indices)
    starting_edges = edge_list[i][starting_edges_indices]
    #print(f"starting edges = {starting_edges}")

    if starting_edges.size < 4:
        raise Exception("Not enough edges for 1st facet start point")

    if starting_edges.size > 4:
        raise Exception("Too many edges for 1st facet start point") 

    #np.where(starting_edges[1,:] != start_point)[0][0]
    possible_starting_vector_1 = new_mesh.vertices[np.where(starting_edges[0,:] != start_point)[0][0]] - new_mesh.vertices[start_point]
    possible_starting_vector_2 = new_mesh.vertices[np.where(starting_edges[1,:] != start_point)[0][0]] - new_mesh.vertices[start_point]

    #find the cross product of the starting vectors
    starting_edges_cross = np.cross(possible_starting_vector_1,possible_starting_vector_2)

    #make sure that the order of the vectors goes so that the cross product is in line with the starting normal
    if np.dot(starting_edges_cross,facet_group_1_normal) > 0:
        #print("group 1 picked")
        processed_edges.append(starting_edges_indices[0])
        current_vertex = starting_edges[0][np.where(starting_edges[0,:] != start_point)[0][0]]
    else:
        #print("group 2 picked")
        processed_edges.append(starting_edges_indices[1])
        #print("np.where(starting_edges[1,:] != start_point) = " + str(np.where(starting_edges[1,:] != start_point)))
        current_vertex = starting_edges[1][np.where(starting_edges[1,:] != start_point)[0][0]]

    #print(f"current_vertex = {current_vertex}" )

    #now iterate through number of 
    for z in range(1,edge_list[i][:,0].size):
        if current_vertex == start_point:
            raise Exception("Start vertex reached before processed all of edges")

        #get the next edge
        counter = 0
        next_vertex = -1
        for j,edg in enumerate(edge_list[i]):
            #print("edg = " + str(edg))
            #print("processed_edges = " + str(processed_edges))
            if current_vertex in edg and j not in processed_edges:
                current_edge_index = j
                if edg[0] != current_vertex:
                    next_vertex = edg[0]
                else:
                    next_vertex = edg[1]


                counter += 1
                if counter >= 2:
                    raise Exception(f"More than 2 edges possibilities for {current_vertex}")

        #make sure the next vertex was found
        if next_vertex <= -1:
            raise Exception(f"No next vertex was found for {current_vertex} ")

        #if found next vertex
        processed_edges.append(current_edge_index)
        edge_order.append(current_vertex)

        current_vertex = next_vertex


    edge_order_list.append(edge_order)
    print(f"edge_{i}_order = {edge_order}")




start_point = 2
edge_0_order = [2, 3, 4, 1]
start_point = 8
edge_1_order = [8, 9, 10, 11, 12, 5, 6, 7]


In [11]:
"""
Now stitch the mesh back together

1) Pick the index with the most number of points
2) Divide the the higher number of faces by the lower and
    store the remainder (dividend)
3) Start a counter=0 and loop through indices of smaller one
store current_partner_index = 0


Inside loop:
a. If not the first face in index:
   Add triangle of previous_smaller, current_smaller, current_bigger
a2. if first just do same thing but previous smaller is wrapped around
b. For dividend + int(current_index < remainder) loop through and add faces of
    traingle of current_smaller, current_bigger, next_bigger

make a trimesh that has those vertices and those faces

"""

'\nNow stitch the mesh back together\n\n1) Pick the index with the most number of points\n2) Divide the the higher number of faces by the lower and\n    store the remainder (dividend)\n3) Start a counter=0 and loop through indices of smaller one\nstore current_partner_index = 0\n\n\nInside loop:\na. If not the first face in index:\n   Add triangle of previous_smaller, current_smaller, current_bigger\na2. if first just do same thing but previous smaller is wrapped around\nb. For dividend + int(current_index < remainder) loop through and add faces of\n    traingle of current_smaller, current_bigger, next_bigger\n\nmake a trimesh that has those vertices and those faces\n\n'

In [24]:
lengths_of_boundaries = [len(x) for x in edge_order_list]
bigger = lengths_of_boundaries.index(max(lengths_of_boundaries))
smaller = 1-bigger

print(f"index of bigger facets = {bigger}\nindex of smaller facets = {smaller}",)

dividend = int(lengths_of_boundaries[bigger]/lengths_of_boundaries[smaller])
remainder = lengths_of_boundaries[bigger] - int(lengths_of_boundaries[bigger]/lengths_of_boundaries[smaller])*lengths_of_boundaries[smaller]

print(f"dividend = {dividend}, remainder = {remainder}")


new_faces = []
current_bigger = 0
for i,current_smaller in enumerate(edge_order_list[smaller]):
    print("current_smaller =" + str(current_smaller))
    if i == 0:
        new_faces.append([current_smaller,edge_order_list[smaller][-1],edge_order_list[bigger][current_bigger]])
    else:
        new_faces.append([current_smaller,edge_order_list[smaller][i-1],edge_order_list[bigger][current_bigger]])
    
    for j in range(0,dividend + int(2<remainder)):
        if current_bigger > len(edge_order_list[bigger]):
            raise Exception("Somehow rapped around too much")
                                
        if current_bigger >= len(edge_order_list[bigger])-1:
            next_bigger = 0
        else:
            next_bigger = current_bigger+1
        
        new_faces.append([current_smaller,edge_order_list[bigger][current_bigger],
                            edge_order_list[bigger][next_bigger]])
        
        current_bigger += 1

print("new_faces = " + str(new_faces))



index of bigger facets = 1
index of smaller facets = 0
dividend = 2, remainder = 0
current_smaller =2
current_smaller =3
current_smaller =4
current_smaller =1
new_faces = [[2, 1, 8], [2, 8, 9], [2, 9, 10], [3, 2, 10], [3, 10, 11], [3, 11, 12], [4, 3, 12], [4, 12, 5], [4, 5, 6], [1, 4, 6], [1, 6, 7], [1, 7, 8]]


In [25]:
#create a new mesh with the new_faces
stitch_mesh = trimesh.Trimesh()

In [56]:
stitch_mesh.vertices = new_mesh.vertices
stitch_mesh.faces = np.array(new_faces)
#stitch_mesh.show()

In [57]:
stitch_mesh.faces

TrackedArray([[ 2,  1,  8],
              [ 2,  8,  9],
              [ 2,  9, 10],
              [ 3,  2, 10],
              [ 3, 10, 11],
              [ 3, 11, 12],
              [ 4,  3, 12],
              [ 4, 12,  5],
              [ 4,  5,  6],
              [ 1,  4,  6],
              [ 1,  6,  7],
              [ 1,  7,  8]])

In [58]:
stitch_mesh.vertices

TrackedArray([[  0.,   0.,   7.],
              [ 10.,   0.,   0.],
              [  0.,  10.,   0.],
              [-10.,   0.,   0.],
              [  0., -10.,   0.],
              [  5.,   0.,   7.],
              [  5.,   5.,   7.],
              [  0.,   5.,   7.],
              [ -5.,   5.,   7.],
              [ -5.,   0.,   7.],
              [ -5.,  -5.,   7.],
              [  0.,  -5.,   7.],
              [  5.,  -5.,   7.]])

In [59]:
trimesh.repair.fix_inversion(stitch_mesh)
trimesh.repair.fix_winding(stitch_mesh)
trimesh.repair.fix_normals(stitch_mesh)
stitch_mesh.show()

In [63]:
#test that new mesh is now closed
new_mesh.faces = np.vstack([new_mesh.faces, stitch_mesh.faces])

In [65]:
from print_trimesh import print_trimesh
print_trimesh(new_mesh,"stitched_boxes.off")

# Testing the creating of the mesh with faces

In [52]:
test_mesh = trimesh.Trimesh()
test_mesh.faces = np.array([[0,1,2],[0,2,3]]) #shows both faces
test_mesh.faces = np.array([[0,2,1],[0,2,3]]) #only shows 2nd one face
test_mesh.faces = np.array([[0,2,1],[0,3,2]]) #shows no faces
test_mesh.vertices = np.array([
   [0,0,5],
    [0,5,0],
    [-5,0,0],
    [0,-5,0]
])

In [53]:
test_mesh.show()

In [54]:
#try repairing the mesh so that it now shows some
trimesh.repair.fix_inversion(test_mesh)
trimesh.repair.fix_winding(test_mesh)
trimesh.repair.fix_normals(test_mesh)

In [55]:
test_mesh.show()

# Testing how to delete parts of mesh

In [90]:
#can just get the submesh of the mesh without the vertices i want 


total_faces = np.linspace(0,len(new_mesh.faces)-1,len(new_mesh.faces)).astype("int")
facet_faces = np.linspace(0,9,10).astype("int")
faces_to_keep = set(total_faces).difference(set(facet_faces))
faces_to_keep
                           


{10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21}

In [91]:
#get the submesh
just_stitching = new_mesh.submesh([list(faces_to_keep)])[0]
print_trimesh(just_stitching,"just_sides.off")

# Code on how to create boundaries with pinch points
# BUT DON'T HAVE TO WORRY ABOUT PINCH POINTS ANYMORE

In [ ]:
def compute_angle(base_point,point_1,base_normal):
    angle = np.arccos(np.dot(base_point,point_1)/(np.linalg.norm(base_point)*np.linalg.norm(point_1)))
    cross = np.cross(base_point,point_1)/(np.linalg.norm(base_point)*np.linalg.norm(point_1))
    
    if(np.dot(base_normal,cross)) < 0:
        angle = 2*np.pi-angle
        
    return angle
    

In [ ]:
#Example of how to use the compute angle function
base_point = np.array([10,0,0])
point_1 = np.array([5,5,1])
point_2 = np.array([5,-5,1])
regular_normal = np.array([0,0,1])

angle_1 = compute_angle(base_point,point_1,regular_normal)
angle_2 = compute_angle(base_point,point_2,regular_normal)

print(angle_1,angle_2)

In [ ]:
"""organize the order of vertices to do the stitching method in
Things to keep in mind: 
1) Keep going not only until hit the start loop, but until hit the start loop and all edges have been processed:
2) Always go with the lower angle option at branch points


Pseudo code
1) Retrieve starting point 
2) For the starting point: 
a. Get all the edges that have that point
b. Go with the first edge in the list as the current edge
c. Store the starting point as the previous point
d. Store the mating point as the current point
e. Store the edge in processed edges list
f. Store sequence of points (point could possibly be repeated)

2) Start loop that will go for the number of edges that exist in the facet
a. Get all the edges that have that current point in them
b. Get list of possible mating points
c. Go with the point that has the lowest angle between line connecting:
    point - current_point
    current_point - previous_point
    
    AND this edge can't have already been processed
    
-- keep going until same number of edges in processed list as there started in number of edges

-- if not end up at start point then error

Result: Have

"""

#organize the first loop
start_edge = edge_0_starting_point
edges_list = edge_0
current_edge = start_edge

ordered_edges = []
for i in range(0,len(edges_list)):
    temp_edge = edge_0[edge_0[:,0] == current_edge]
    
    #get the edge that hasn't already been used
    
    ordered_edges.append(temp_edge)
    current_edge = temp_edge[1]
    
    
next_edge



In [ ]:
"""
Pseudocode
https://stackoverflow.com/questions/5188561/signed-angle-between-two-3d-vectors-with-same-origin-within-the-same-plane

angle = acos(dotProduct(Va.normalize(), Vb.normalize()));
cross = crossProduct(Va, Vb);
if (dotProduct(Vn, cross) < 0) { // Or > 0
  angle = -angle;
}

"""


In [ ]:
"""Possible Solution for finding angle
1) Define plane with either:
a. Base vector and cross product of base vector and normal
b. 3 points from the base face
2) Project all points onto that plane to work with

3) When ordering the points and have possibel two paths that havent passed:
- Calculate the signed angle between them
- Always go with the path that has the least angle between them

"""

regular_normal = np.array([0,0,1])

example_vectors = np.array([[10,0,0],[5,5,1],[-5,5,1]]) #will show the difference in angle between 45 and 135
example_vectors = np.array([[10,0,0],[5,5,1],[5,-5,1]]) #will not show a difference in angle between 45 and 135
 
positive_dot = np.arccos(np.dot(example_vectors[0],example_vectors[1])/(np.linalg.norm(example_vectors[0])*np.linalg.norm(example_vectors[1])))
negative_dot = np.arccos(np.dot(example_vectors[0],example_vectors[2])/(np.linalg.norm(example_vectors[0])*np.linalg.norm(example_vectors[2])))


positive_cross = np.cross(example_vectors[0],example_vectors[1])/(np.linalg.norm(example_vectors[1])*np.linalg.norm(example_vectors[0]))
negative_cross = np.cross(example_vectors[0],example_vectors[2])/(np.linalg.norm(example_vectors[2])*np.linalg.norm(example_vectors[0]))
print(positive_dot,negative_dot)

#get the dot product
positive_dot_cross = np.dot(positive_cross,regular_normal)
negative_dot_cross = np.dot(negative_cross,regular_normal)

print(positive_dot_cross,negative_dot_cross)


""" if want to get the angles
positive_dot = np.arccos(np.dot(example_vectors[0],example_vectors[1])/(np.linalg.norm(example_vectors[0])*np.linalg.norm(example_vectors[1])))*180/np.pi
negative_dot = np.arccos(np.dot(example_vectors[0],example_vectors[2])/(np.linalg.norm(example_vectors[0])*np.linalg.norm(example_vectors[2])))*180/np.pi

"""



In [ ]:
#find the cross product between the normal
new_mesh.face_normals[1]

positive_cross = np.linalg.norm(np.cross(example_vectors[0],example_vectors[1]))/np.linalg.norm(example_vectors[1])
negative_cross = np.linalg.norm(np.cross(example_vectors[0],example_vectors[2]))/np.linalg.norm(example_vectors[2])

print(positive_cross,negative_cross)